# Phase 1

## Preprocessing

In [1]:
import warnings

import pandas as pd
import numpy as np
from dateutil.parser import parse
from tmdbv3api import Movie
from tmdbv3api import TMDb

import keys  # TMDb API key file

warnings.filterwarnings("ignore")

##### Reading the train datasets

In [ ]:
# reading csv files
movies_base = pd.read_csv('datasets/1/train/movies-revenue.csv')
movies_director = pd.read_csv('datasets/1/train/movie-director.csv')

##### Checking nulls

In [ ]:
movies_base.isnull().sum()

##### Joining directors dataset into the movies (main) dataset

In [ ]:
# Merging dataframes; how = 'left' := use only keys from left frame.
df = pd.merge(movies_base, movies_director, on='name', how='left')

# displaying result
df.head()

##### Filling the null directors

In [ ]:
df.isnull().sum()

In [ ]:
df.describe().T

In [ ]:
df.dtypes

In [ ]:
# Formatting the release_date to match the TMDb date format

for i, movie in df.iterrows():
    # Parse date from a string and return a datetime.datetime
    release_date = parse(movie['release_date'])
    # Remove the time from it reducing it to just the date
    release_date = release_date.date()

    # Parser doesn't do well with dates prior to the 80s
    # Correcting the dates newer than this year to a century earlier
    if release_date.year > 2023:
        release_date = release_date.replace(year=release_date.year - 100)

    # Editing the value at the original dataframe
    df.at[i, 'release_date'] = release_date

df['release_date'] = pd.to_datetime(df['release_date'])
df.head(n=10)

##### Initializing TMDb API

In [2]:
# Using the TMDb to fill out the missing director from the original dataset

# https://github.com/AnthonyBloomer/tmdbv3api
# https://developers.themoviedb.org/3/getting-started/introduction

# Creating a base class instance from the api library
tmdb = TMDb()
tmdb.api_key = keys.tmdb_key
tmdb.language = 'en'
tmdb.debug = True


##### Using the TMDb API to fill missing directors via the movie title and its release date
Since the director name is still not encoded, the director's popularity score will replace it.

In [ ]:
# Creating a Movie instance to search by the movie details
movie = Movie()

for i, mov in df.iterrows():
    search = movie.search(mov['name'])  # Search by the movie title
    for res in search:
        try:
            # Confirming the search results by the release date year
            mov_date_str = str(mov['release_date'])
            if res['release_date'][:4] == mov_date_str[:4]:
                # Extracting the director from the movie credits
                for member in movie.credits(res.id)['crew']:
                    if member['job'] == 'Director':
                        # Editing the value at the original dataframe
                        df.at[i, 'director'] = member['popularity']
                        break
                break
        except BaseException as error:
            print('An exception occurred: {}'.format(error) + " " + mov['name'])

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

##### Cleaning up the revenue to extract the float value

In [ ]:
df['revenue'] = df['revenue'].str.replace(',', '').str.replace('$', '').astype('float')
print(df.dtypes)
df.head()

##### Encoding the MPAA Rating and Genre to indicator variables

In [ ]:
# Fill the nulls with the mode
df['director'] = df['director'].fillna(value=df['director'].median())
df['genre'] = df['genre'].fillna(value=df['genre'].mode()[0])
df['MPAA_rating'] = df['MPAA_rating'].fillna(value=df['MPAA_rating'].mode()[0])
df.isnull().sum()

In [ ]:
# Preprocessing genre and MPAA_rating

df = pd.get_dummies(df, columns=["MPAA_rating"], prefix=["rating_is"])
df = pd.get_dummies(df, columns=["genre"], prefix=["genre_is"])
df.head()

##### Encoding the release date to a scalar

In [ ]:
def date_to_float(dt):
    # Calculating the months and days
    calc = (((dt.month - 1) * 30) + dt.day) / 365
    # Adding calc to the years
    return dt.year + calc

##### Splitting date to days and months and then converting it to a scalar via `date_to_float()`

In [ ]:
df['month'] = df['release_date'].dt.month
df['day'] = df['release_date'].dt.day
df['season'] = df['release_date'].dt.quarter
df['release_date'] = df['release_date'].apply(date_to_float)
df.head()

##### Adjusting the revenue to inflation

In [3]:
df = pd.read_csv('datasets/1/train/preprocessed.csv')

df['budget'] = np.nan
df['runtime'] = np.nan

movie = Movie()

# extract the budget and runtime from the TMDb API
for i, mov in df.iterrows():
    search = movie.search(mov['name'])  # Search by the movie title
    for res in search:
        try:
            # Confirming the search results by the release date year
            mov_date_str = str(mov['release_date'])
            if int(res['release_date'][:4]) == int(mov_date_str[:4]):
                # Editing the value at the original dataframe
                detail = movie.details(res.id)
                df.at[i, 'budget'] = detail['budget']
                df.at[i, 'runtime'] = detail['runtime']
                break
        except BaseException as error:
            print('An exception occurred: {}'.format(error) + " " + mov['name'])

df.head()

,name,release_date,revenue,director,rating_is_G,rating_is_Not Rated,rating_is_PG,rating_is_PG-13,rating_is_R,genre_is_Action,...,genre_is_Horror,genre_is_Musical,genre_is_Romantic Comedy,genre_is_Thriller/Suspense,genre_is_Western,month,day,season,budget,runtime
0,Recess: School's Out,2001.126027,54656124.0,0.814,1,0,0,0,0,0,...,0,0,0,0,0,2,16,1,10000000.0,82.0
1,D2: The Mighty Ducks,1994.232877,94226333.0,1.048,0,0,1,0,0,0,...,0,0,0,0,0,3,25,1,0.0,106.0
2,Home on the Range,2004.252055,67910166.0,1.975,0,0,1,0,0,0,...,0,0,0,0,0,4,2,2,110000000.0,76.0
3,Young Black Stallion,2003.972603,9254344.0,4.610,1,0,0,0,0,0,...,0,0,0,0,0,12,25,4,0.0,49.0
4,What's Love Got to Do With It,1993.435616,79618610.0,0.946,0,0,0,0,1,0,...,0,0,0,0,0,6,9,2,15000000.0,118.0


In [5]:
import cpi

# Update the CPI data
# cpi.update()

# Adjust the revenue to inflation using cpi
df['revenue'] = df.apply(lambda x: cpi.inflate(x['revenue'],
                                               int(x.release_date)), axis=1)
df['budget'] = df.apply(lambda x: cpi.inflate(x['budget'],
                                              int(x.release_date)), axis=1)

##### Feature Engineering

In [6]:
df_animation = pd.read_csv('datasets/1/train/movie-voice-actors.csv')

# Remove duplicates on name column
df_animation = df_animation.drop_duplicates(subset=['name'], keep='first')

# Add column is_animation
df_animation['is_animation'] = 1

# Drop other columns that are not needed
df_animation = df_animation.drop(['voice_actor', 'character'], axis=1)

df_animation.head()

,name,is_animation
0,The Aristocats,1
1,The Return of Jafar,1
2,Aladdin,1
3,The Hunchback of Notre Dame,1
4,The Little Mermaid,1


##### Merging the animation dataset with the main dataset

In [7]:
# Merge the animation dataset with the main dataset
df = pd.merge(df, df_animation, on='name', how='left')

# Fill the nulls with 0
df['is_animation'] = df['is_animation'].fillna(value=0)

df.head()

,name,release_date,revenue,director,rating_is_G,rating_is_Not Rated,rating_is_PG,rating_is_PG-13,rating_is_R,genre_is_Action,...,genre_is_Musical,genre_is_Romantic Comedy,genre_is_Thriller/Suspense,genre_is_Western,month,day,season,budget,runtime,is_animation
0,Recess: School's Out,2001.126027,9.031840e+07,0.814,1,0,0,0,0,0,...,0,0,0,0,2,16,1,1.652484e+07,82.0,0.0
1,D2: The Mighty Ducks,1994.232877,1.860716e+08,1.048,0,0,1,0,0,0,...,0,0,0,0,3,25,1,0.000000e+00,106.0,0.0
2,Home on the Range,2004.252055,1.052104e+08,1.975,0,0,1,0,0,0,...,0,0,0,0,4,2,2,1.704185e+08,76.0,1.0
3,Young Black Stallion,2003.972603,1.471919e+07,4.610,1,0,0,0,0,0,...,0,0,0,0,12,25,4,0.000000e+00,49.0,0.0
4,What's Love Got to Do With It,1993.435616,1.612511e+08,0.946,0,0,0,0,1,0,...,0,0,0,0,6,9,2,3.037941e+07,118.0,0.0


##### Saving the preprocessed dataset

In [8]:
df.to_csv('datasets/1/train/preprocessed_added.csv', index=False)

##### Fitting the Linear Regression model

In [9]:
from scipy.stats import stats
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

df = pd.read_csv('datasets/1/train/preprocessed_added.csv')
df = df.drop(['name'], axis=1)

df['budget'] = df['budget'].replace(0, df['budget'].median())
df['budget'] = df['budget'].fillna(value=df['budget'].median())

df['runtime'] = df['runtime'].replace(0, df['runtime'].median())
df['runtime'] = df['runtime'].fillna(value=df['runtime'].median())

print(df.shape)

# drop outliers from the dataset via z score
df = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]

print(df.shape)

# plot correlation matrix
corr = df.corr()

# plot correlation with revenue
rev_corr = abs(corr['revenue']).sort_values(ascending=False)
rev_corr


release_date                    0
revenue                         0
director                        0
rating_is_G                     0
rating_is_Not Rated             0
rating_is_PG                    0
rating_is_PG-13                 0
rating_is_R                     0
genre_is_Action                 0
genre_is_Adventure              0
genre_is_Black Comedy           0
genre_is_Comedy                 0
genre_is_Concert/Performance    0
genre_is_Documentary            0
genre_is_Drama                  0
genre_is_Horror                 0
genre_is_Musical                0
genre_is_Romantic Comedy        0
genre_is_Thriller/Suspense      0
genre_is_Western                0
month                           0
day                             0
season                          0
budget                          0
runtime                         0
is_animation                    0
dtype: int64
(463, 26)
(304, 26)


revenue                         1.000000
budget                          0.404898
director                        0.212792
genre_is_Adventure              0.202678
genre_is_Drama                  0.174183
month                           0.173107
rating_is_G                     0.157768
season                          0.156895
rating_is_R                     0.132848
release_date                    0.116818
runtime                         0.106150
rating_is_PG-13                 0.058739
rating_is_PG                    0.050063
day                             0.015025
genre_is_Comedy                 0.013137
rating_is_Not Rated                  NaN
genre_is_Action                      NaN
genre_is_Black Comedy                NaN
genre_is_Concert/Performance         NaN
genre_is_Documentary                 NaN
genre_is_Horror                      NaN
genre_is_Musical                     NaN
genre_is_Romantic Comedy             NaN
genre_is_Thriller/Suspense           NaN
genre_is_Western

In [5]:
# drop columns with low correlation
df = df.drop(corr[corr['revenue'] < 0.1].index, axis=1)

print(df.shape)

X = df.drop(['revenue'], axis=1)
Y = df['revenue']

# Feature Scaling
sc = MinMaxScaler()
X = sc.fit_transform(X)

# Splitting the dataset into the Training set and Test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

# Fitting Multiple Linear Regression to the Training set
regressor = linear_model.LinearRegression()
regressor.fit(X_train, Y_train)

# Predicting the Test set results
Y_pred = regressor.predict(X_test)

# The coefficients
print('Coefficients: \n', regressor.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % metrics.mean_squared_error(Y_test, Y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % metrics.r2_score(Y_test, Y_pred))


(304, 19)
Coefficients: 
 [ 8.14553375e+07  4.32163274e+07 -1.19209290e-07 -2.98023224e-08
  4.77783013e+06 -5.96046448e-08  2.98023224e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  1.19423486e+08 -4.08194348e+07  3.18175758e+08
 -2.46915983e+07  0.00000000e+00]
Mean squared error: 20888175887351192.00
Variance score: 0.28


##### Testing the model

In [ ]:
# Loading the test dataset

movies_base_test = pd.read_csv('datasets/1/test/movies-revenue.csv')
movies_director_test = pd.read_csv('datasets/1/test/movie-director.csv')

# Merging the two datasets
df_test = pd.merge(movies_base_test, movies_director_test, on='name', how='left')

# Preprocessing the test dataset
df_test['release_date'] = pd.to_datetime(df_test['release_date'])
df_test['release_date'] = df_test['release_date'].apply(date_to_float)
df_test['month'] = df_test['release_date'].dt.month
df_test['day'] = df_test['release_date'].dt.day
df_test['season'] = df_test['release_date'].dt.quarter
df_test['release_date'] = df_test['release_date'].apply(date_to_float)
df_test = pd.get_dummies(df_test, columns=["MPAA_rating"], prefix=["rating_is"])
df_test = pd.get_dummies(df_test, columns=["genre"], prefix=["genre_is"])
df_test = pd.merge(df_test, df_animation, on='name', how='left')
df_test['is_animation'] = df_test['is_animation'].fillna(value=0)
df_test = df_test.drop(['name'], axis=1)

df_test = df_test.drop(['revenue'], axis=1)
Y_test = df['revenue']

# Feature Scaling
X_test = sc.transform(df_test)

# Predicting the Test set results
y_pred = regressor.predict(X_test)

# The mean squared error
print('Mean squared error: %.2f'
      % metrics.mean_squared_error(Y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % metrics.r2_score(Y_test, y_pred))

